In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.6 MB/s 
     |████████████████████████████████| 7.6 MB 27.1 MB/s 
     |████████████████████████████████| 182 kB 22.8 MB/s 


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

#data processing
import re, string
import nltk

from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split


#Naive Bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

#transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

#keras
import tensorflow as tf
from tensorflow import keras


#metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

#set seed for reproducibility
seed=42

#set style for plots
sns.set_style("whitegrid")
sns.despine()
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlepad=10)

<Figure size 432x288 with 0 Axes>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('drive/MyDrive/Dataset/twitter_sentiment_data.csv')
df

,sentiment,message,tweetid
0,-1,@tiniebeany climate change is an interesting h...,792927353886371840
1,1,RT @NatGeoChannel: Watch #BeforeTheFlood right...,793124211518832641
2,1,Fabulous! Leonardo #DiCaprio's film on #climat...,793124402388832256
3,1,RT @Mick_Fanning: Just watched this amazing do...,793124635873275904
4,2,"RT @cnalive: Pranita Biswasi, a Lutheran from ...",793125156185137153
...,...,...,...
43938,1,"Dear @realDonaldTrump,\nYeah right. Human Medi...",791307031919550464
43939,1,What will your respective parties do to preven...,791316857403936768
43940,2,RT @MikkiL: UN Poll Shows Climate Change Is th...,791357509101621249
43941,0,RT @taehbeingextra: i still can$q$t believe th...,791390042136641537


In [5]:
df1 = pd.read_csv('drive/MyDrive/Dataset/Twitter-sentiment-self-drive-DFE.csv', encoding="latin-1")
df1

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,sentiment,sentiment:confidence,our_id,sentiment_gold,sentiment_gold_reason,text
0,724227031,True,golden,236,NaN,5,0.7579,10001,5\n4,Author is excited about the development of the...,Two places I'd invest all my money if I could:...
1,724227032,True,golden,231,NaN,5,0.8775,10002,5\n4,Author is excited that driverless cars will be...,Awesome! Google driverless cars will help the ...
2,724227033,True,golden,233,NaN,2,0.6805,10003,2\n1,The author is skeptical of the safety and reli...,If Google maps can't keep up with road constru...
3,724227034,True,golden,240,NaN,2,0.8820,10004,2\n1,The author is skeptical of the project's value.,Autonomous cars seem way overhyped given the t...
4,724227035,True,golden,240,NaN,3,1.0000,10005,3,Author is making an observation without expres...,Just saw Google self-driving car on I-34. It w...
...,...,...,...,...,...,...,...,...,...,...,...
7151,726942339,True,golden,65,NaN,not_relevant,0.5616,10073,not_relevant,Trains (metros) are not relevant to the focus ...,Washington DC metro trains are all self-drivin...
7152,726942340,True,golden,71,NaN,not_relevant,0.7293,10074,not_relevant,Outside the scope of the task,I think #DARPA works on autonomous drone techn...
7153,726942341,True,golden,63,NaN,not_relevant,0.8854,10075,not_relevant,Outside the scope of the task,I love the idea of driving myself to the denti...
7154,726942342,True,golden,67,NaN,not_relevant,0.8431,10076,not_relevant,Outside the scope of the task,Who likes cars these days? No me. Too expensiv...


In [6]:
df11 = df1[df1['sentiment'] != 'not_relevant']

In [7]:
df2 = df11['sentiment'].replace(['4', '5'], 1)
df2 = df2.replace(['3'], 0)
df2 = df2.replace(['1', '2'], -1)

In [8]:
df11['sentiment'] = df2

<ipython-input-8-5abbb10dfd9b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11['sentiment'] = df2


In [9]:
df11.reset_index(drop=True, inplace=True)

In [10]:
df11['sentiment'].value_counts()

 0    4245
 1    1903
-1     795
Name: sentiment, dtype: int64

In [6]:
df = df[df.sentiment != 2]

In [11]:
df11.drop_duplicates(subset=['text'], inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [12]:
df11.reset_index(drop=True, inplace=True)

In [74]:
df11

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,sentiment,sentiment:confidence,our_id,sentiment_gold,sentiment_gold_reason,text
0,724227031,True,golden,236,NaN,1,0.7579,10001,5\n4,Author is excited about the development of the...,Two places I'd invest all my money if I could:...
1,724227032,True,golden,231,NaN,1,0.8775,10002,5\n4,Author is excited that driverless cars will be...,Awesome! Google driverless cars will help the ...
2,724227033,True,golden,233,NaN,-1,0.6805,10003,2\n1,The author is skeptical of the safety and reli...,If Google maps can't keep up with road constru...
3,724227034,True,golden,240,NaN,-1,0.8820,10004,2\n1,The author is skeptical of the project's value.,Autonomous cars seem way overhyped given the t...
4,724227035,True,golden,240,NaN,0,1.0000,10005,3,Author is making an observation without expres...,Just saw Google self-driving car on I-34. It w...
...,...,...,...,...,...,...,...,...,...,...,...
6928,726942329,True,golden,60,NaN,0,0.9275,10063,3,"Simply states a fact, not an opinion",Florida is one of a few states that allows tes...
6929,726942330,True,golden,61,NaN,0,0.8449,10064,3,"Simply states a fact, not an opinion",Many states still considering allowing self-dr...
6930,726942331,True,golden,58,NaN,0,0.7704,10065,3,"Simply states a fact, not an opinion",Heading to a tech conference which will focus ...
6931,726942332,True,golden,74,NaN,0,0.8874,10066,3,"Simply states a fact, not an opinion",Fully autonomous vehicles are one of many auto...


In [8]:
df.reset_index(drop=True, inplace=True)

In [13]:
def data_cleaning(line):
  regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
  clean_text = regrex_pattern.sub(r'', line)
  clean_text = clean_text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
  clean_text = re.sub(r"(?:\@|https?\://)\S+", "", clean_text) #remove links and mentions
  clean_text = re.sub(r'[^\x00-\x7f]',r'', clean_text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
  banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
  table = str.maketrans('', '', banned_list)
  clean_text = clean_text.translate(table)
  new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', clean_text)) #remove last hashtags
  new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
  new_tweet2 = re.sub("\s\s+" , " ", new_tweet2)
  clean_text = new_tweet2.replace('$', '')
  clean_text = clean_text.replace('#', '')
  return clean_text

In [14]:
df11['clean_text'] = df11['text'].apply(lambda func: data_cleaning(func))

<ipython-input-14-83c4836323a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11['clean_text'] = df11['text'].apply(lambda func: data_cleaning(func))


In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [16]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [17]:
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [18]:
lbls = [-1, 0, 1]

In [19]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.9 MB/s 


In [20]:
df11['encoded_tweet'] = df11['clean_text'].apply(lambda func: tokenizer(func, return_tensors='pt'))

<ipython-input-20-773cdc319043>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11['encoded_tweet'] = df11['clean_text'].apply(lambda func: tokenizer(func, return_tensors='pt'))


In [21]:
df11['predicted'] = ''

<ipython-input-21-44373b45c83b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11['predicted'] = ''


In [22]:
for idx, val in enumerate(df11['encoded_tweet']):
  enc = df11['encoded_tweet'][idx]
  output = model(**enc)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  map_idx = scores.argmax(axis=0)
  lbl = lbls[map_idx]
  df11['predicted'].iloc[idx] = lbl

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [23]:
df11

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,sentiment,sentiment:confidence,our_id,sentiment_gold,sentiment_gold_reason,text,clean_text,encoded_tweet,predicted
0,724227031,True,golden,236,NaN,1,0.7579,10001,5\n4,Author is excited about the development of the...,Two places I'd invest all my money if I could:...,two places id invest all my money if i could 3...,"[input_ids, attention_mask]",1
1,724227032,True,golden,231,NaN,1,0.8775,10002,5\n4,Author is excited that driverless cars will be...,Awesome! Google driverless cars will help the ...,awesome google driverless cars will help the b...,"[input_ids, attention_mask]",1
2,724227033,True,golden,233,NaN,-1,0.6805,10003,2\n1,The author is skeptical of the safety and reli...,If Google maps can't keep up with road constru...,if google maps cant keep up with road construc...,"[input_ids, attention_mask]",-1
3,724227034,True,golden,240,NaN,-1,0.8820,10004,2\n1,The author is skeptical of the project's value.,Autonomous cars seem way overhyped given the t...,autonomous cars seem way overhyped given the t...,"[input_ids, attention_mask]",1
4,724227035,True,golden,240,NaN,0,1.0000,10005,3,Author is making an observation without expres...,Just saw Google self-driving car on I-34. It w...,just saw google selfdriving car on i34 it was ...,"[input_ids, attention_mask]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6928,726942329,True,golden,60,NaN,0,0.9275,10063,3,"Simply states a fact, not an opinion",Florida is one of a few states that allows tes...,florida is one of a few states that allows tes...,"[input_ids, attention_mask]",1
6929,726942330,True,golden,61,NaN,0,0.8449,10064,3,"Simply states a fact, not an opinion",Many states still considering allowing self-dr...,many states still considering allowing selfdri...,"[input_ids, attention_mask]",0
6930,726942331,True,golden,58,NaN,0,0.7704,10065,3,"Simply states a fact, not an opinion",Heading to a tech conference which will focus ...,heading to a tech conference which will focus ...,"[input_ids, attention_mask]",1
6931,726942332,True,golden,74,NaN,0,0.8874,10066,3,"Simply states a fact, not an opinion",Fully autonomous vehicles are one of many auto...,fully autonomous vehicles are one of many auto...,"[input_ids, attention_mask]",1


In [87]:
df11.to_csv('take2.csv')

### XLM-R

In [ ]:
xlmr = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(xlmr)
tokenizer = AutoTokenizer.from_pretrained(xlmr)
config = AutoConfig.from_pretrained(xlmr)

In [ ]:
df11['encoded_tweet'] = df11['clean_text'].apply(lambda func: tokenizer(func, return_tensors='pt'))

In [ ]:
df11['predicted'] = ''

In [ ]:
for idx, val in enumerate(df11['encoded_tweet']):
  enc = df11['encoded_tweet'][idx]
  output = model(**enc)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  map_idx = scores.argmax(axis=0)
  lbl = lbls[map_idx]
  df11['predicted'].iloc[idx] = lbl

In [ ]:
df11

### Happy Transformer

In [24]:
!pip install happytransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 2.1 MB/s 
     |████████████████████████████████| 451 kB 10.2 MB/s 
     |████████████████████████████████| 132 kB 43.9 MB/s 
     |████████████████████████████████| 212 kB 41.5 MB/s 
     |████████████████████████████████| 127 kB 47.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [25]:
from happytransformer import HappyTextClassification

In [26]:
happy_tc = HappyTextClassification(model_type="DISTILBERT", model_name="distilbert-base-uncased-finetuned-sst-2-english", num_labels=2)

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [37]:
result = happy_tc.classify_text(df11['clean_text'][1])

In [38]:
result

TextClassificationResult(label='POSITIVE', score=0.9855216145515442)